In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, activations
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import Adam,SGD
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import cv2 as cv2
import sklearn.metrics as sklearn
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

In [2]:

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
!unzip gdrive/My\ Drive/Archive/archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: archive/train/Benign/6078.jpg  
  inflating: __MACOSX/archive/train/Benign/._6078.jpg  
  inflating: archive/train/Benign/1717.jpg  
  inflating: __MACOSX/archive/train/Benign/._1717.jpg  
  inflating: archive/train/Benign/283.jpg  
  inflating: __MACOSX/archive/train/Benign/._283.jpg  
  inflating: archive/train/Benign/4647.jpg  
  inflating: __MACOSX/archive/train/Benign/._4647.jpg  
  inflating: archive/train/Benign/3128.jpg  
  inflating: __MACOSX/archive/train/Benign/._3128.jpg  
  inflating: archive/train/Benign/2236.jpg  
  inflating: __MACOSX/archive/train/Benign/._2236.jpg  
  inflating: archive/train/Benign/5559.jpg  
  inflating: __MACOSX/archive/train/Benign/._5559.jpg  
  inflating: archive/train/Benign/6050.jpg  
  inflating: __MACOSX/archive/train/Benign/._6050.jpg  
  inflating: archive/train/Benign/1059.jpg  
  inflating: __MACOSX/archive/train/Benign/._1059.jpg  
  inflating: archive/train/Benign/3896.jpg

In [4]:
train_path = '/content/archive/train'
test_path = '/content/archive/test'

In [5]:
class_names = ['Benign',"Malignant"]
class_labels = {}
for i, classes in enumerate(class_names, start=0):
  class_labels[classes] = i

print(class_labels)

{'Benign': 0, 'Malignant': 1}


In [6]:
train_data = []
train_labels = []

image_size = (150, 150)

for folder in os.listdir(train_path):
    print("In folder: {}".format(folder))
    for file in os.listdir(os.path.join(train_path, folder)):
        image_path = os.path.join(train_path, folder, file)
        image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, image_size)  # Resize the image using image_size
        train_data.append(image)
        train_labels.append(class_labels[folder])

train_data = np.array(train_data, dtype='float32')
train_labels = np.array(train_labels, dtype='int32')

train_data = train_data / 255.0

In folder: Malignant
In folder: Benign


In [7]:
test_data = []
test_labels = []

for folder in os.listdir(test_path):
  print("In folder: {}".format(folder))
  for file in os.listdir(os.path.join(test_path, folder)):
    image_path = os.path.join(test_path, folder, file)
    image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size)
    test_data.append(image)
    test_labels.append(class_labels[folder])

test_data = np.array(test_data, dtype='float32')
test_labels = np.array(test_labels, dtype='int32')

test_data = test_data/255.0

In folder: Malignant
In folder: Benign


In [8]:
train_data.shape

(11879, 150, 150, 3)

In [9]:
model = tf.keras.Sequential([
        # 1st Convolutional Layer
        tf.keras.layers.Resizing(227,227, interpolation="bilinear", input_shape = train_data.shape[1:]),
        tf.keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227, 227, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)),

        # 2nd Convolutional Layer
        tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)),

        # 3rd Convolutional Layer
        tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),

        # 4th Convolutional Layer
        tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),

        # 5th Convolutional Layer
        tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)),

        # Flattening
        tf.keras.layers.Flatten(),

        # 1st Fully Connected Layer
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),

        # 2nd Fully Connected Layer
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),

        # 3rd Fully Connected Layer
        tf.keras.layers.Dense(2, activation='sigmoid')
    ])

In [10]:
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 227, 227, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization (Batch  (None, 55, 55, 96)        384       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 27, 27, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_1 (Bat  (None, 27, 27, 256)       1

In [11]:
call_back = EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [12]:
from keras.utils import to_categorical

train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)

In [13]:
past_data = model.fit(
    train_data,
    train_labels_one_hot,
    callbacks=call_back,
    batch_size=32,
    epochs=10,
    validation_split = 0.2
  )

Epoch 1/10
297/297 [==============================] - 33s 73ms/step - loss: 1.6499 - accuracy: 0.7698 - val_loss: 0.2184 - val_accuracy: 0.9444
Epoch 2/10
297/297 [==============================] - 17s 59ms/step - loss: 0.4494 - accuracy: 0.8185 - val_loss: 0.5690 - val_accuracy: 0.8594
Epoch 3/10
297/297 [==============================] - 18s 59ms/step - loss: 0.4445 - accuracy: 0.8212 - val_loss: 0.5447 - val_accuracy: 0.7449
Epoch 4/10
297/297 [==============================] - 17s 59ms/step - loss: 0.3991 - accuracy: 0.8316 - val_loss: 1.0564 - val_accuracy: 0.3321
Epoch 5/10
297/297 [==============================] - 17s 58ms/step - loss: 0.3939 - accuracy: 0.8384 - val_loss: 0.4557 - val_accuracy: 0.8594
Epoch 6/10
297/297 [==============================] - 17s 58ms/step - loss: 0.3757 - accuracy: 0.8486 - val_loss: 0.8399 - val_accuracy: 0.5025
Epoch 6: early stopping


In [14]:
test_labels_binary = np.argmax(
    test_labels_one_hot,
    axis=1
  )
loss, acc = model.evaluate(
    test_data,
    test_labels_one_hot,
    verbose=2
  )
print(f'Test accuracy: {acc:.4f}')

63/63 - 1s - loss: 0.4762 - accuracy: 0.8215 - 1s/epoch - 24ms/step
Test accuracy: 0.8215


In [15]:
y_pred = np.argmax(model.predict(test_data), axis=-1)

63/63 [==============================] - 2s 17ms/step


In [16]:
score=accuracy_score(test_labels,y_pred)
print(score)

0.8215


In [17]:
score=f1_score(test_labels,y_pred)
print(score)

0.8425231583590648


In [18]:
score=recall_score(test_labels,y_pred)
print(score)

0.955


In [19]:
score=precision_score(test_labels,y_pred)
print(score)

0.7537490134175217
